# Install Feast on Kubernetes with the Feast Operator
## Objective

Provide a reference implementation of a runbook to deploy a Feast environment on a Kubernetes cluster using [Kind](https://kind.sigs.k8s.io/docs/user/quick-start) and the [Feast Operator](../../infra/feast-operator/).

## Prerequisites
* Kubernetes Cluster
* [kubectl](https://kubernetes.io/docs/tasks/tools/#kubectl) Kubernetes CLI tool.

## Install the Feast Operator

In [4]:
## Use this install command from a release branch (e.g. 'v0.46-branch')
!kubectl apply -f ../../infra/feast-operator/dist/install.yaml

## OR, for the latest code/builds, use one the following commands from the 'master' branch
# !make -C ../../infra/feast-operator install deploy IMG=quay.io/feastdev-ci/feast-operator:develop FS_IMG=quay.io/feastdev-ci/feature-server:develop
# !make -C ../../infra/feast-operator install deploy IMG=quay.io/feastdev-ci/feast-operator:$(git rev-parse HEAD) FS_IMG=quay.io/feastdev-ci/feature-server:$(git rev-parse HEAD)

!kubectl wait --for=condition=available --timeout=5m deployment/feast-operator-controller-manager -n feast-operator-system

namespace/feast-operator-system created
customresourcedefinition.apiextensions.k8s.io/featurestores.feast.dev created
serviceaccount/feast-operator-controller-manager created
role.rbac.authorization.k8s.io/feast-operator-leader-election-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-featurestore-editor-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-featurestore-viewer-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-manager-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-metrics-auth-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-metrics-reader created
rolebinding.rbac.authorization.k8s.io/feast-operator-leader-election-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/feast-operator-manager-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/feast-operator-metrics-auth-rolebinding created
service/feast-operator-controller-manager-metrics-service created
deployment.ap

## Install the Feast services via FeatureStore CR
Next, we'll use the running Feast Operator to install the feast services. Before doing that it is important to understand basic understanding of operator support of Volumes and volumeMounts and how to mount TLS certificates.

### Mounting TLS Certificates with Volumes in Feast Operator  

The Feast operator supports **volumes** and **volumeMounts**, allowing you to mount TLS certificates onto a pod. This approach provides flexibility in how you mount these files, supporting different Kubernetes resources such as **Secrets, ConfigMaps,** and **Persistent Volumes (PVs).**  

#### Example: Mounting Certificates Using Kubernetes Secrets  

In this example, we demonstrate how to mount TLS certificates using **Kubernetes Secrets** that were created in a previous notebook.  

#### PostgreSQL Connection Parameters  

When connecting to PostgreSQL with TLS, some important parameters in the connection URL are:  

- **`sslrootcert`** – Specifies the path to the **CA certificate** file used to validate trusted certificates.  
- **`sslcert`** – Provides the client certificate for **mutual TLS (mTLS) encryption**.  
- **`sslkey`** – Specifies the private key for the client certificate.  

If mutual TLS authentication is not required, you can **omit** the `sslcert` and `sslkey` parameters. However, the `sslrootcert` parameter is still necessary for validating server certificates.  


<b><font color=red> Note: Please deploy either option 1 or 2 only. Don't deploy both of them at the same time to avoid conflicts in the lateral steps. </font></b>

**Option 1: Directly Setting the CA Certificate Path**  

In this approach, we specify the CA certificate path directly in the Feast PostgreSQL URL using the `sslrootcert` parameter.  

You can refer to the `v1_featurestore_postgres_db_volumes_tls.yaml` file for the complete configuration details.  

In [17]:
!kubectl apply -f ../../infra/feast-operator/config/samples/v1_featurestore_postgres_db_volumes_tls.yaml --namespace=feast

secret/postgres-secret created
secret/feast-data-stores created
featurestore.feast.dev/sample-db-ssl created


**Option 2: Using an Environment Variable for the CA Certificate**  

In this approach, you define the CA certificate path as an environment variable. You can refer to the `v1_featurestore_postgres_tls_volumes_ca_env.yaml` file for the complete configuration details.   

```bash
FEAST_CA_CERT_FILE_PATH=<path-to-ca-cert>


In [26]:
!kubectl apply -f ../../infra/feast-operator/config/samples/v1_featurestore_postgres_tls_volumes_ca_env.yaml --namespace=feast

secret/postgres-secret created
secret/feast-data-stores created
featurestore.feast.dev/sample-db-ssl created


## Validate the running FeatureStore deployment
Validate the deployment status.

In [31]:
!kubectl wait --for=condition=available --timeout=8m deployment/feast-sample-db-ssl -n feast
!kubectl get all

deployment.apps/feast-sample-db-ssl condition met
NAME                                     READY   STATUS    RESTARTS   AGE
pod/feast-sample-db-ssl-86b47d54-hclb9   1/1     Running   0          27s
pod/postgresql-0                         1/1     Running   0          13h

NAME                                 TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
service/feast-sample-db-ssl-online   ClusterIP   10.96.61.65   <none>        80/TCP     27s
service/postgresql                   ClusterIP   10.96.228.3   <none>        5432/TCP   13h
service/postgresql-hl                ClusterIP   None          <none>        5432/TCP   13h

NAME                                  READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/feast-sample-db-ssl   1/1     1            1           27s

NAME                                           DESIRED   CURRENT   READY   AGE
replicaset.apps/feast-sample-db-ssl-86b47d54   1         1         1       27s

NAME                          READY   AGE
statef

Validate that the FeatureStore CR is in a `Ready` state.

In [32]:
!kubectl get feast

NAME            STATUS   AGE
sample-db-ssl   Ready    33s


Verify that the DB includes the expected tables.

In [33]:
!kubectl exec postgresql-0 -- env PGPASSWORD=password psql -U admin -d feast -c '\dt'

Defaulted container "postgresql" out of: postgresql, init-chmod-data (init)
                               List of relations
 Schema |                         Name                         | Type  | Owner 
--------+------------------------------------------------------+-------+-------
 public | data_sources                                         | table | admin
 public | entities                                             | table | admin
 public | feast_metadata                                       | table | admin
 public | feature_services                                     | table | admin
 public | feature_views                                        | table | admin
 public | managed_infra                                        | table | admin
 public | on_demand_feature_views                              | table | admin
 public | permissions                                          | table | admin
 public | postgres_tls_sample_env_ca_driver_hourly_stats       | table | admin
 pub

Verify the client `feature_store.yaml` and create the sample feature store definitions.

In [34]:
!kubectl exec deploy/feast-sample-db-ssl -c online -- cat feature_store.yaml
!kubectl exec deploy/feast-sample-db-ssl -c online -- feast apply
print(" Feast apply is completed. You can go to next step.")

project: postgres_tls_sample_env_ca
provider: local
offline_store:
    host: ${POSTGRES_HOST}
    type: postgres
    port: 5432
    database: ${POSTGRES_DB}
    db_schema: public
    password: ${POSTGRES_PASSWORD}
    sslcert_path: /var/lib/postgresql/certs/tls.crt
    sslkey_path: /var/lib/postgresql/certs/tls.key
    sslmode: verify-full
    sslrootcert_path: system
    user: ${POSTGRES_USER}
online_store:
    type: postgres
    database: ${POSTGRES_DB}
    db_schema: public
    host: ${POSTGRES_HOST}
    password: ${POSTGRES_PASSWORD}
    port: 5432
    sslcert_path: /var/lib/postgresql/certs/tls.crt
    sslkey_path: /var/lib/postgresql/certs/tls.key
    sslmode: verify-full
    sslrootcert_path: system
    user: ${POSTGRES_USER}
registry:
    path: postgresql+psycopg://${POSTGRES_USER}:${POSTGRES_PASSWORD}@${POSTGRES_HOST}:5432/${POSTGRES_DB}?sslmode=verify-full&sslrootcert=system&sslcert=/var/lib/postgresql/certs/tls.crt&sslkey=/var/lib/postgresql/certs/tls.key
    registry_type: 

List the registered feast projects & feature views.

In [35]:
!kubectl exec deploy/feast-sample-db-ssl -c online -- feast projects list
!kubectl exec deploy/feast-sample-db-ssl -c online -- feast feature-views list

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/opt/app-root/src/sdk/python/feast/feature_view.py:48: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity '__dummy'.
  DUMMY_ENTITY = Entity(
/opt/app-root/src/sdk/python/feast/entity.py:173: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity 'driver'.
  entity = cls(
/opt/app-root/src/sdk/python/feast/entity.py:173: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity '__dummy'.
  entity = cls(
NAME                        DESCRIPTION                      TAGS    OWNER
postgres_tls_sample                                          {}
postgres_tls_sample_env_ca  A project for driver statistics  {}
<jemalloc>: MADV_DONTNEED does not work (memset will be used

Finally, let's verify the feast version.

In [36]:
!kubectl exec deployment/feast-sample-db-ssl -c online -- feast version

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/opt/app-root/src/sdk/python/feast/feature_view.py:48: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity '__dummy'.
  DUMMY_ENTITY = Entity(
Feast SDK Version: "0.1.dev1+g6c92447.d20250213"
